In [2]:
import json
from tqdm.auto import tqdm 
from elasticsearch import Elasticsearch

In [3]:
with open('../documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
model = SentenceTransformer("all-mpnet-base-v2")

In [8]:
sentence = 'This is a sample sentence'

In [9]:
# Get the embeddings
embeddings = model.encode(sentence)

In [10]:
embeddings

array([ 3.38363014e-02, -8.39075446e-02, -3.01863700e-02, -1.50362467e-02,
       -8.36342722e-02,  2.67099999e-02, -9.17344168e-03,  3.30599165e-03,
       -4.73221280e-02, -1.27413385e-02,  8.20042714e-02,  2.05611717e-02,
        1.98337473e-02, -3.27860676e-02,  1.76552609e-02, -6.64511770e-02,
        5.29538579e-02,  2.12427862e-02, -4.35398668e-02,  2.42592283e-02,
        3.84504953e-03,  1.64461248e-02,  3.46780270e-02,  2.50417404e-02,
        2.40206886e-02, -3.28407064e-02,  5.68538159e-03, -2.87025254e-02,
       -6.85417186e-03, -3.71620469e-02,  1.50978956e-02, -1.02682021e-02,
       -3.70135996e-03, -1.02162898e-01,  1.71727072e-06, -3.04966490e-03,
       -8.88465717e-03, -1.37526821e-02, -5.74573278e-02,  3.62290032e-02,
        4.53157490e-03,  6.52434677e-02, -1.65316591e-03,  5.79803400e-02,
       -1.97080392e-02,  2.01019440e-02,  5.05361222e-02,  3.05137467e-02,
       -3.38397026e-02,  5.72957657e-02, -4.07247664e-03, -1.28480233e-02,
       -4.37915660e-02, -

In [11]:
embeddings[0]

0.0338363

In [12]:
embeddings.shape

(768,)

In [13]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [14]:
operations[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.035703521221876144,
  -0.06891413033008575,
  -0.04448336735367775,
  0.006250121630728245,
  -0.04224712774157524,
  -0.00645399047061801,
  0.027548134326934814,
  -0.018610365688800812,
  -0.05203752964735031,
  -0.0024046744219958782,
  0.05745451897382736,
  -0.015592566691339016,
  0.015488805249333382,
  0.09870167821645737,
  0.015649665147066116,
  -0

In [35]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [36]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [37]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [38]:
def elastic_search(query, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [39]:
elastic_search(
    query="I just discovered the course. Can I still join?",
    course="data-engineering-zoomcamp"
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'text_vector': [-0.057206664234399796,
   0.019887413829565048,
   -0.024336013942956924,
   -0.010076597332954407,
   0.03898910805583,
   0.010869272984564304,
   0.026468904688954353,
   -0.0339190848171711,
   0.029106318950653076,
   -0.049129944294691086,
   0.03300593048334122,
   -0.06733977794647217,
   -0.021426716819405556,
   -0.013197499327361584,
   -0.035638730973005295,
   0.06667866557836533,
   -0.04194784536957741,
   -0.05011485517024994,
   -0.05269114300608635,
   -0.04796300083398819,
   -0.06410785019397736,
   -0.0028072763234376907,
   -0.046105168759822845,
   0.025319106876

In [40]:
import pandas as pd

In [41]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')


In [42]:
ground_truth = df_ground_truth.to_dict(orient='records')


In [43]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/4627 [00:00<?, ?it/s]

In [44]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [45]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [47]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

In [48]:
def minsearch_search(query, course):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

In [49]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/4627 [00:00<?, ?it/s]

In [50]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [51]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['course']))


  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.7395720769397017, 'mrr': 0.6029788920106625}

In [52]:
evaluate(ground_truth, lambda q: minsearch_search(q['question'], q['course']))


  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.7722066133563864, 'mrr': 0.661454506159499}

In [59]:
# Perform the search query to get the document with the highest score
response = es_client.search(
    index=index_name,
    body={
        "query": {
            "match_all": {}
        },
        "size": 1,  # We only need the top result
        "sort": [
            {
                "_score": {
                    "order": "desc"
                }
            }
        ]
    }
)

# Extract the ID and score of the document with the highest score
if response['hits']['hits']:
    top_hit = response['hits']['hits'][0]
    doc_id = top_hit['_id']
    score = top_hit['_score']
    print(f"Document ID with the highest score: {doc_id}, Score: {score}")
else:
    print("No documents found.")



Document ID with the highest score: RkC7vJABhYzG9Dnw0EB8, Score: 1.0
